In [1]:
import torch.nn as nn
import torch
from torchsummary import summary

In [2]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResidualBlock, self).__init__()
        self.conv_path = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                      kernel_size=3, padding=1, stride=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=out_channels, out_channels=out_channels,
                      kernel_size=3, padding=1, stride=1))

        self.skip_path = nn.Identity()
        self.out_act = nn.ReLU()

    def forward(self, x):
        out = self.conv_path(x)
        out += self.skip_path(x)
        out = self.out_act(out)
        return out

In [3]:
class ResidualBlockDown(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResidualBlockDown, self).__init__()

        self.conv_path = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                      kernel_size=3, padding=1, stride=2),
            nn.ReLU(),
            nn.Conv2d(in_channels=out_channels, out_channels=out_channels,
                      kernel_size=3, padding=1, stride=1))

        self.skip_path = nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                                   kernel_size=1, padding=0, stride=2)

        self.out_act = nn.ReLU()

    def forward(self, x):
        out = self.conv_path(x)
        out += self.skip_path(x)
        out = self.out_act(out)
        return out

In [4]:
class ResNet34(nn.Module):
    def __init__(self, in_channels):
        super(ResNet34, self).__init__()
        # 1
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=64,
                               kernel_size=7, stride=2, padding=3)
        self.conv1_act = nn.ReLU()

        # 2
        self.conv2 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            ResidualBlock(in_channels=64, out_channels=64),
            ResidualBlock(in_channels=64, out_channels=64),
            ResidualBlock(in_channels=64, out_channels=64))
        self.conv2_act = nn.ReLU()

        # 3
        self.conv3 = nn.Sequential(
            ResidualBlockDown(in_channels=64, out_channels=128),
            ResidualBlock(in_channels=128, out_channels=128),
            ResidualBlock(in_channels=128, out_channels=128),
            ResidualBlock(in_channels=128, out_channels=128))
        self.conv3_act = nn.ReLU()

        # 4
        self.conv4 = nn.Sequential(
            ResidualBlockDown(in_channels=128, out_channels=256),
            ResidualBlock(in_channels=256, out_channels=256),
            ResidualBlock(in_channels=256, out_channels=256),
            ResidualBlock(in_channels=256, out_channels=256),
            ResidualBlock(in_channels=256, out_channels=256),
            ResidualBlock(in_channels=256, out_channels=256))
        self.conv4_act = nn.ReLU()

        # 5
        self.conv5 = nn.Sequential(
            ResidualBlockDown(in_channels=256, out_channels=512),
            ResidualBlock(in_channels=512, out_channels=512),
            ResidualBlock(in_channels=512, out_channels=512))
        self.conv5_act = nn.ReLU()

        # 6
        self.avgpool = nn.AvgPool2d(kernel_size=7, stride=1)
        self.fc = nn.Linear(in_features=512, out_features=1000)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv1_act(x)
        x = self.conv2(x)
        x = self.conv2_act(x)
        x = self.conv3(x)
        x = self.conv3_act(x)
        x = self.conv4(x)
        x = self.conv4_act(x)
        x = self.conv5(x)
        x = self.conv5_act(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x


def run_resnet():
    input_tensor = torch.randn(size=(32, 3, 224, 224))
    model = ResNet34(in_channels=3)
    pred = model(input_tensor)
    print(pred.shape)

run_resnet()

torch.Size([32, 1000])


In [5]:
class ResidualBlock_Total(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock_Total,self).__init__()

        self.conv_path = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, stride=stride),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, stride=1),
        )

        self.skip_path = nn.Identity() if stride == 1 else nn.Conv2d(in_channels, out_channels, kernel_size=1, padding=0, stride=stride)

        self.out_act = nn.ReLU()
        
    def forward(self,x):
        out = self.conv_path(x)
        out += self.skip_path(x)
        out = self.out_act(out)

        return out

In [6]:
class ResNet34_re(nn.Module):
    def __init__(self, in_channels):
        super(ResNet34_re, self).__init__()
        # 1
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=64,
                               kernel_size=7, stride=2, padding=3)
        self.conv1_act = nn.ReLU()

        # 2
        self.conv2 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            ResidualBlock_Total(in_channels=64, out_channels=64),
            ResidualBlock_Total(in_channels=64, out_channels=64),
            ResidualBlock_Total(in_channels=64, out_channels=64))
        self.conv2_act = nn.ReLU()

        # 3
        self.conv3 = nn.Sequential(
            ResidualBlock_Total(in_channels=64, out_channels=128),
            ResidualBlock_Total(in_channels=128, out_channels=128),
            ResidualBlock_Total(in_channels=128, out_channels=128),
            ResidualBlock_Total(in_channels=128, out_channels=128))
        self.conv3_act = nn.ReLU()

        # 4
        self.conv4 = nn.Sequential(
            ResidualBlock_Total(in_channels=128, out_channels=256),
            ResidualBlock_Total(in_channels=256, out_channels=256),
            ResidualBlock_Total(in_channels=256, out_channels=256),
            ResidualBlock_Total(in_channels=256, out_channels=256),
            ResidualBlock_Total(in_channels=256, out_channels=256),
            ResidualBlock_Total(in_channels=256, out_channels=256))
        self.conv4_act = nn.ReLU()

        # 5
        self.conv5 = nn.Sequential(
            ResidualBlock_Total(in_channels=256, out_channels=512),
            ResidualBlock_Total(in_channels=512, out_channels=512),
            ResidualBlock_Total(in_channels=512, out_channels=512))
        self.conv5_act = nn.ReLU()

        # 6
        self.avgpool = nn.AvgPool2d(kernel_size=7, stride=1)
        self.fc = nn.Linear(in_features=512, out_features=1000)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv1_act(x)
        x = self.conv2(x)
        x = self.conv2_act(x)
        x = self.conv3(x)
        x = self.conv3_act(x)
        x = self.conv4(x)
        x = self.conv4_act(x)
        x = self.conv5(x)
        x = self.conv5_act(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x


def run_resnet():
    input_tensor = torch.randn(size=(32, 3, 224, 224))
    model = ResNet34(in_channels=3)
    pred = model(input_tensor)
    print(pred.shape)

run_resnet()

torch.Size([32, 1000])
